# Animal Crossing Database Queries

This notebook demonstrates SQLite queries on the Animal Crossing SQLite Database. This is merely a showcase of some of the capabilities of the database and is not the web application, that is found in app.py. It will create and seed the database and demonstrate the following types of queries:

- Two queries involving four or more relations
- One query involving joins
- One query using an aggregate function
- Three queries using subqueries in a non-trivial way
- Two queries using grouping, include the use of HAVING
- One query using set operations

## Setup

First, we'll import the necessary libraries and set up the SQLite database connection:


In [617]:
import sqlite3
import pandas as pd
import os

# Create an in-memory SQLite database
conn = sqlite3.connect('./acnh.db')
cursor = conn.cursor()

# Helper function to execute SQL queries and display results as DataFrames
def execute_query(query, params=None):
    if params:
        result = pd.read_sql_query(query, conn, params=params)
    else:
        result = pd.read_sql_query(query, conn)
    return result

## Creating Database Tables from DDL File

In [618]:
# Read the DDL file
with open('animal-crossing-ddl.sql', 'r') as file:
    ddl_sql = file.read()

# Execute the DDL statements
cursor.executescript(ddl_sql)
conn.commit()

print("Database tables created successfully.")

Database tables created successfully.


## Loading Data from Seed File

In [619]:
# Read the seed file
with open('animal-crossing-seed.sql', 'r') as file:
    seed_sql = file.read()

# Execute the seed statements
cursor.executescript(seed_sql)
conn.commit()

print("Data loaded successfully.")

Data loaded successfully.


## Verifying the Data

In [620]:
# view all items
data = execute_query("SELECT * FROM ITEM")
print("Item Table:")
data

Item Table:


,ItemID,Name,Category
0,art001,Mona Lisa,ARTWORK
1,art002,Starry Night,ARTWORK
2,art003,The Scream,ARTWORK
3,art004,The Birth of Venus,ARTWORK
4,art005,Girl with a Pearl Earring,ARTWORK
...,...,...,...
68,craft001,Wood,CRAFTING
69,craft002,Iron Nugget,CRAFTING
70,craft003,Clay,CRAFTING
71,craft004,Stone,CRAFTING


In [621]:
# view all villagers
data = execute_query("SELECT * FROM VILLAGER")
print("Villager_static Table:")
data

Villager_static Table:


,VillagerID,Name,Personality,Species
0,vill001,Raymond,Smug,Cat
1,vill002,Audie,Peppy,Wolf
2,vill003,Sherb,Lazy,Goat
3,vill004,Judy,Snooty,Cub
4,vill005,Marshal,Smug,Squirrel
5,vill006,Ankha,Snooty,Cat
6,vill007,Stitches,Lazy,Cub
7,vill008,Beau,Lazy,Deer
8,vill009,Pietro,Smug,Sheep
9,vill010,Diana,Snooty,Deer


In [622]:
# view all users
data = execute_query("SELECT * FROM USER")
print("Users:")
data

Users:


,UserID,Username,Password
0,1,ExampleUser,password123


In [623]:
data = execute_query("SELECT * FROM USER_ITEM")
print("User owned items:")
data

User owned items:


,UserID,ItemID,QuantityOwned
0,1,fish001,2
1,1,bug001,1
2,1,art001,1
3,1,fossil001,1
4,1,flower001,3
5,1,flower002,5
6,1,sea001,1
7,1,sea003,1
8,1,sea005,2


In [624]:
data = execute_query("SELECT * FROM ISLAND_VILLAGER")
print("User villagers:")
data

User villagers:


,VillagerID,UserID,TalkedToday
0,vill011,1,0
1,vill012,1,0
2,vill013,1,0
3,vill014,1,1
4,vill015,1,1


## SQLite Queries

Queries involving four or more relations:

In [625]:
# this shows the Sea Critters currently owned by the User with id = 1.
query = """
SELECT i.Name AS SeaCritterName, sc.ShadowSize, sc.Behavior, ui.QuantityOwned
FROM USER_ITEM ui
JOIN ITEM i ON ui.ItemID = i.ItemID
JOIN SEA_CRITTER sc ON ui.ItemID = sc.ItemID
JOIN USER u ON ui.UserID = u.UserID
WHERE u.UserID = '1' AND i.Category = 'SEA_CRITTER' AND ui.QuantityOwned = 1;
"""

result = execute_query(query)
print("Results:")
result


Results:


,SeaCritterName,ShadowSize,Behavior,QuantityOwned
0,Sea Urchin,Medium,Fast,1
1,Sea Pig,Small,Stationary,1


In [626]:
# this shows the Fish currently owned by the user with id = 1
query = """
SELECT i.Name AS FishName, f.Location, ui.QuantityOwned
FROM USER_ITEM ui
JOIN ITEM i ON ui.ItemID = i.ItemID
JOIN FISH f ON ui.ItemID = f.ItemID
JOIN USER u ON ui.UserID = u.UserID
WHERE u.UserID = '1' AND i.Category = 'FISH' AND ui.QuantityOwned >= 1;
"""

result = execute_query(query)
print("Results:")
result

Results:


,FishName,Location,QuantityOwned
0,Bluegill,River,2


Query involving joins

In [627]:
# this shows the Villagers currently on the user's island with id = 1
query = """
SELECT vs.Name AS VillagerName, vs.Personality, vs.Species, v.TalkedToday
FROM VILLAGER vs
JOIN ISLAND_VILLAGER v ON v.VillagerID = vs.VillagerID
WHERE v.UserID = '1'
"""

result = execute_query(query)
print("Results:")
result

Results:


,VillagerName,Personality,Species,TalkedToday
0,Maple,Normal,Cub,0
1,Fauna,Normal,Deer,0
2,Meringue,Normal,Rhino,0
3,Poppy,Normal,Squirrel,1
4,Bob,Lazy,Cat,1


Query using an aggregate function

In [628]:
# this shows the number of villagers that have been talked to by the user with id = 1
query = """
SELECT COUNT(*) as NumberVillagersTalkedTo
FROM ISLAND_VILLAGER
WHERE UserID = '1' AND TalkedToday = 1
"""

result = execute_query(query)
print("Results:")
result

Results:


,NumberVillagersTalkedTo
0,2


Queries using subqueries in a non-trivial way

In [629]:
# this shows the villagers with the snooty personality on the user with id = 1's island
query = """
SELECT v.Name
FROM VILLAGER v
JOIN ISLAND_VILLAGER iv ON v.VillagerID = iv.VillagerID
WHERE iv.UserID = '1'
AND v.Species IN (
    SELECT DISTINCT Species
    FROM VILLAGER
    WHERE Personality = 'Snooty'
);
"""

result = execute_query(query)
print("Results:")
result

Results:


,Name
0,Maple
1,Fauna
2,Bob


In [630]:
# this shows the flowers owned by user with id = 1 where more than 1 flower color is owned
query = """
SELECT i.Name AS FlowerName
FROM USER_ITEM ui
JOIN ITEM i ON ui.ItemID = i.ItemID
WHERE ui.UserID = '1' AND i.Category = 'FLOWER'
AND i.ItemID IN (
    SELECT fc.ItemID
    FROM FLOWER_COLOR fc
    GROUP BY fc.ItemID
    HAVING COUNT(fc.Color) > 2
);
"""

result = execute_query(query)
print("Results:")
result

Results:


,FlowerName
0,Rose
1,Tulip


In [631]:
# this shows the fossils in the user with id = 1's wishlist that have a wanted quantity over 2
query = """
SELECT i.Name AS FossilName
FROM ITEM i
WHERE i.Category = 'FOSSIL'
AND i.ItemID IN (
    SELECT w.ItemID
    FROM WISHLIST w
    WHERE w.UserID = '1'
    AND WishlistQuantity > 2
);
"""

result = execute_query(query)
print("Results:")
result

Results:


,FossilName
0,Sabertooth Skull
1,Brachiosaurus Tail


Queries using grouping, include the use of HAVING

In [632]:
# this shows the items owned by the user with id = 1 with a quantity owned greater than 1
query = """
SELECT i.Name, QuantityOwned
FROM USER_ITEM ui
JOIN ITEM i ON i.ItemID = ui.ItemID
WHERE ui.UserID = '1'
GROUP BY QuantityOwned
HAVING QuantityOwned > 1
ORDER BY QuantityOwned DESC
"""

result = execute_query(query)
print("Results:")
result

Results:


,Name,QuantityOwned
0,Tulip,5
1,Rose,3
2,Bluegill,2


In [633]:
# this shows the villagers on the user with id = 1 that have the Lazy personality
query = """
SELECT v.Name
FROM VILLAGER v
JOIN ISLAND_VILLAGER vi ON vi.VillagerID = v.VillagerID
WHERE vi.UserID = '1'
GROUP BY v.Name
HAVING Personality = 'Lazy'
"""

result = execute_query(query)
print("Results:")
result

Results:


,Name
0,Bob


Query using set operations

In [634]:
# this shows the item ids of items not yet collected by the user with id = 1
query = """
SELECT ItemID
FROM ITEM
EXCEPT
SELECT ItemID
FROM USER_ITEM
WHERE UserID = '1'
"""

result = execute_query(query)
print("Results:")
result

Results:


,ItemID
0,art002
1,art003
2,art004
3,art005
4,art006
...,...
59,sea006
60,sea007
61,sea008
62,sea009


In [635]:
conn.close()